In [ ]:
import numpy as np
from PIL import Image
import scipy.fftpack
import os

def read_image(image_path):
    # Read image and convert it to YCbCr (YUV) color space.
    image = Image.open(image_path)
    image_ycbcr = image.convert('YCbCr')
    img_array = np.array(image_ycbcr, dtype=float)
    return img_array

def chroma_subsampling(Cb, Cr):
    # 4:2:0 Chroma Subsampling
    Cb_subsampled = Cb[::2, ::2]
    Cr_subsampled = Cr[::2, ::2]
    return Cb_subsampled, Cr_subsampled



def main(): 
    image_path = "OriginalImage.png"
    img_channels = read_image(image_path)
    
    Y = img_channels[:, :, 0]
    Cb = img_channels[:, :, 1]
    Cr = img_channels[:, :, 2]
    
    # 4:2:0 Chroma Subsampling
    Cb_sampled, Cr_sampled = chroma_subsampling(Cb, Cr)
    
    
    
    
